## Commonsense

In [1]:
from cot import Collection
coll = Collection.load_thoughtsource_100(load_pregenerated_cots=True)
coll.select_generated_cots(cot_trigger = "kojima-01", api_service='openai',model = 'text-davinci-003')

/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/robertpraas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
coll['strategy_qa']['train'][0]

{'id': '1b29d402c3e17cb3b435',
 'ref_id': '',
 'question': 'Is a pound sterling valuable?',
 'type': 'bool',
 'choices': ['True', 'False'],
 'context': '',
 'cot': ['A pound sterling is fiat money.',
  'Fiat money is backed by government decree and has no intrinsic value.',
  'One pound sterling is worth about 1.24 US dollars by May of 2020.'],
 'answer': ['False'],
 'generated_cot': [],
 'feedback': None}

In [6]:
from cot import Collection
coll = Collection.load_thoughtsource_100(load_pregenerated_cots=True)
coll.select_generated_cots(cot_trigger = "kojima-01", api_service='openai',model = 'text-davinci-003')
coll.select_generated_cots(answers="False")
coll.unload_datasets(["med_qa","medmc_qa","strategy_qa",'open_book_qa','worldtree'])
coll
#coll.dump("cs_davinci.json")


| Name           | Train   |   Valid | Test   |
|----------------|---------|---------|--------|
| commonsense_qa | -       |     100 | -      |

Not loaded: ['aqua', 'asdiv', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'medmc_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp', 'worldtree']

In [ ]:
"""Went through Annotator"""

In [9]:
annotated = Collection.from_json("cs_davinci_annotated.json")


| Name           | Train   |   Valid | Test   |
|----------------|---------|---------|--------|
| commonsense_qa | -       |     100 | -      |

Not loaded: ['aqua', 'asdiv', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'medmc_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp', 'worldtree']

In [80]:
"""Moved to annotated files folder"""

ann_1 = Collection.from_json("annotated_ob_davinci.json") #open book
ann_2 = Collection.from_json("annotated_st_davinci.json") #strategu
ann_3 = Collection.from_json("annotated_wt_davinci.json")
ann_4 = Collection.from_json("cs_davinci_annotated.json")

In [37]:
def select_annotated(dataset,split,selection):
    i= 0
    lst = []
    for element in selection[dataset][split]:
        if len(element['generated_cot'])>0:
            if len(element['generated_cot'][0]['annotations'])>0 :
                lst.append(i)
        i+=1
    return lst

In [ ]:
ann_1 = Collection.from_json("annotated_ob_davinci.json") #open book
ann_2 = Collection.from_json("annotated_st_davinci.json") #strategu
ann_3 = Collection.from_json("annotated_wt_davinci.json")
ann_4 = Collection.from_json("cs_davinci_annotated.json")

In [81]:
ann_1['open_book_qa']['test'] = ann_1['open_book_qa']['test'].select(select_annotated('open_book_qa','test',ann_1))
ann_2['strategy_qa']['train'] = ann_2['strategy_qa']['train'].select(select_annotated('strategy_qa','train',ann_2))
ann_3['worldtree']['test'] = ann_3['worldtree']['test'].select(select_annotated('worldtree','test',ann_3))
ann_4['commonsense_qa']['validation'] = ann_4['commonsense_qa']['validation'].select(select_annotated('commonsense_qa','validation',ann_4))
 annotated_set = ann_4.merge(ann_3).merge(ann_2).merge(ann_1)

In [ ]:
annotated_set.dump('annotated_set.json')

## get gpt-4 scores

In [11]:
from cot import Collection
col = Collection.from_json("../notebooks/thoughtsource_100_openai_chat_gpt-4_None_zhou-01.json")
col.select_generated_cots(api_service='openai_chat',model = 'gpt-4')




In [12]:
col['commonsense_qa']['validation'][0]

{'id': 'b8c0a4703079cf661d7261a60a1bcbff',
 'ref_id': '',
 'question': 'Where would you find magazines along side many other printed works?',
 'type': 'multiplechoice',
 'choices': ['doctor', 'bookstore', 'market', 'train station', 'mortuary'],
 'context': '',
 'cot': ['If one wants various printed reading material, they can go to a bookstore.',
  'All other options are invalid as they do not have wide range of printed reading material.'],
 'answer': ['bookstore'],
 'generated_cot': [{'id': 'e6c02977-8516-484c-849a-3fe640a1aa3b',
   'fragments_version': '0.01',
   'instruction': None,
   'cot_trigger': 'zhou-01',
   'cot_trigger_template': '{instruction}\n\n{question}\n{answer_choices}\n\n{cot_trigger}',
   'prompt_text': '',
   'cot': 'A) doctor - Doctors may have magazines in their waiting rooms, but not many other printed works.\nB) bookstore - Bookstores have a wide variety of printed works, including magazines, books, newspapers, and more.\nC) market - Markets may have some magazi

In [ ]:
#Look at annotated set

In [13]:
from cot import Collection
ann_col = Collection.from_json("../notebooks/internal_documentation/annotated_files/annotated_set.json")
ann_col

| Name           | Train   | Valid   | Test   |
|----------------|---------|---------|--------|
| commonsense_qa | -       | 11      | -      |
| worldtree      | -       | -       | 6      |
| strategy_qa    | 9       | -       | -      |
| open_book_qa   | -       | -       | 10     |

Not loaded: ['aqua', 'asdiv', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'medmc_qa', 'pubmed_qa', 'qed', 'svamp']

In [17]:
id_list = ann_col['commonsense_qa']['validation']['id']

In [25]:
for item in col['commonsense_qa']['validation']:
    if item['id'] in id_list:
        print(item['generated_cot'][0]['answers'][0]['correct_answer'])

# GPT-4 7/11 correct

True
False
False
True
True
False
True
True
False
True
True


In [26]:
id_list_2 = ann_col['worldtree']['test']['id']
id_list_3 = ann_col['strategy_qa']['train']['id']
id_list_4 = ann_col['open_book_qa']['test']['id']

In [27]:
for item in col['worldtree']['test']:
    if item['id'] in id_list_2:
        print(item['generated_cot'][0]['answers'][0]['correct_answer'])

True
True
True
True
True
True


In [28]:
for item in col['strategy_qa']['train']:
    if item['id'] in id_list_3:
        print(item['generated_cot'][0]['answers'][0]['correct_answer'])

False
True
True
True
True
True
False
True
False


In [30]:
for item in col['open_book_qa']['test']:
    if item['id'] in id_list_4:
        print(item['generated_cot'][0]['answers'][0]['correct_answer'])

True
True
True
True
True
False
True
True
True
True


## Get Medical dataset

In [31]:
from cot import Collection
coll = Collection.load_thoughtsource_100(load_pregenerated_cots=True)
coll.select_generated_cots(cot_trigger = "kojima-01", api_service='openai',model = 'text-davinci-003')
coll.select_generated_cots(answers="False")
coll.unload_datasets(["commonsense_qa","strategy_qa",'open_book_qa','worldtree'])

In [32]:
coll

| Name     | Train   | Valid   | Test   |
|----------|---------|---------|--------|
| med_qa   | -       | -       | 100    |
| medmc_qa | -       | 100     | -      |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp', 'worldtree']

In [33]:
i = 0
for item in coll['med_qa']['test']:
    if len(item['generated_cot'])>0:
        i += 1
print(i)

57


In [35]:
i = 0
for item in coll['medmc_qa']['validation']:
    if len(item['generated_cot'])>0:
        i += 1
print(i)

64


In [36]:
coll.dump('medical_incorrect.json')

## Extra annotation

In [38]:
ann_1 = Collection.from_json("../notebooks/internal_documentation/newly_annotated/annotated_ob_davinci.json") #open book
ann_2 = Collection.from_json("../notebooks/internal_documentation/newly_annotated/annotated_st_davinci.json") #strategu
ann_3 = Collection.from_json("../notebooks/internal_documentation/newly_annotated/annotated_wt_davinci.json")
ann_4 = Collection.from_json("../notebooks/internal_documentation/newly_annotated/cs_davinci_annotated.json")


In [ ]:
ann_1['open_book_qa']['test'] = ann_1['open_book_qa']['test'].select(select_annotated('open_book_qa','test',ann_1))
ann_2['strategy_qa']['train'] = ann_2['strategy_qa']['train'].select(select_annotated('strategy_qa','train',ann_2))
ann_3['worldtree']['test'] = ann_3['worldtree']['test'].select(select_annotated('worldtree','test',ann_3))
ann_4['commonsense_qa']['validation'] = ann_4['commonsense_qa']['validation'].select(select_annotated('commonsense_qa','validation',ann_4))
annotated_set = ann_4.merge(ann_3).merge(ann_2).merge(ann_1)

In [41]:
annotated_set

| Name           | Train   | Valid   | Test   |
|----------------|---------|---------|--------|
| commonsense_qa | -       | 12      | -      |
| worldtree      | -       | -       | 7      |
| strategy_qa    | 11      | -       | -      |
| open_book_qa   | -       | -       | 19     |

Not loaded: ['aqua', 'asdiv', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'medmc_qa', 'pubmed_qa', 'qed', 'svamp']